In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import networkx as nx
import music21 as msc
from funciones import *
import sys
import os
from music21 import corpus

env = msc.environment.UserSettings()
#env['musicxmlPath'] = r'C:\Program Files (x86)\Finale NotePad 2012\Finale NotePad.exe' #Path a la aplicacion Finale
#env['musicxmlPath'] = r'C:\Program Files (x86)\MuseScore 2\bin\MuseScore.exe'          #Path a la aplicacion MuseScore 
#env['musicxmlPath']='/usr/bin/mscore'
msc.environment.set('musescoreDirectPNGPath', 'C:\\Program Files (x86)\\MuseScore 2\\bin\\MuseScore.exe')
msc.environment.set('musicxmlPath', 'C:\\Program Files (x86)\\MuseScore 2\\bin\\MuseScore.exe')

#---------------------------------------------------------------------------------------------------------
#            FUNCIONES PARA ANALISIS DE MUSICA:
#---------------------------------------------------------------------------------------------------------
#Lista de funciones:
# f_xml2graph (cancion, nombre_parte=0,modelo='melodia')
# graficar (G, color_map='rainbow',layout='espiral', labels=False)
# ql_2_fig (ql)
# f_motifs_rhytmic (cancion,length,nombre_parte=0)
# f_motifs_tonal (cancion,length,nombre_parte=0)
# f_grado_dist_M (G)
# f_grado_dist_R (G)
# f_tabla (G,nombre)
# f_xml2graph_armonia (cancion, index)
# f_armon (cancion, indexes)
# graficar_armonias_undirected(G, color_map='rainbow',layout='espiral')
# graficar_armonias_directed(Armonias)
# f_dist_escalas (cancion, nombre_parte=0)
# f_full_graph(path)
# f_hierarchy(G)
# f_rewiring_directed(G)
# transitivity_motifs(G)

#-------------------

In [ ]:

musicos_1=['mozart'] #son los más conocidos y que más biblio tienen 

#creo una lista que contenga los grafos con cada artista, por ahora hay solo 2 grafos por artista va a ser un tuple
G=[]

for l, musico in enumerate(musicos_1):
    paths = corpus.getComposer(musico)
    paths=paths[0:20] #corta hasta los primeros 20 temas
    for i, path in enumerate(paths):
        g=f_voices(path, 'melodia')
        


In [ ]:
ls1={'a':5, 'b':1, 'c':2}
#ls3=[inst for g,inst in ls1]
ls2={'a':4, 'b':1, 'c':2}
ls=dict(union(set(ls1),set(ls2)))
#ls4=[inst for g,inst in ls2]
inst1 = [value for value in ls1.values()]
inst2 = [value for value in ls2.values()]
print(inst1)
print(inst2)
print(ls1['a'])
print(ls)
#dif1 = list(set(inst1)-set(inst2))
#dif2 = list(set(inst2)-set(inst1))
#dif=dif1+dif2
#inter=list(set(inst1) & set(inst2))


In [2]:
#toma un xml y separa los grafos de cada voz, devuelve un dict con con cada grafo como .value y la voz como .key
def f_voices(path, modelo='melodia'): #'melodia' 'ritmo' 'armoniaD' y 'armoniaU'
    
    #creo la lista de listas
    g=dict()

    if modelo == 'melodia':
        song = msc.converter.parse(path)
        for i,elem in enumerate(song.parts):#recorro todas las voces
            m=f_xml2graph(path, nombre_parte=i, modelo=modelo); #obtengo el grafo
            g.update({elem.partName:m})  

        
    if modelo == 'ritmo':
        song = msc.converter.parse(path)
        for i,elem in enumerate(song.parts):#recorro todas las voces
            m=f_xml2graph(path, nombre_parte=i, modelo=modelo); #obtengo el grafo
            g.update({elem.partName:m})  

    if modelo == 'armoniaD':
        song = msc.converter.parse(path)
        for i,elem in enumerate(song.parts):#recorro todas las voces
            D,tiempos,U=f_armon (path, voces) #obtengo el grafo
            g.update({elem.partName:m})  

    if modelo == 'armoniaU':
        song = msc.converter.parse(path)
        for i,elem in enumerate(song.parts):#recorro todas las voces
            m=D,tiempos,U=f_armon (path, voces) #obtengo el grafo
            g.update({elem.partName:m})  
            
    return(g)

In [3]:
#Toma dos dict y las mergea cada voz por separado. Devuelve un dict de los grafos correspondientes 
# con la etiqueta de la voz que pertenecian. 

def f_merge(ls1,ls2,modelo='melodia'): #puede ser melodia o ritmo, hay que agregarle 'armonia'

    #H1=nx.DiGraph()
    #H2=nx.DiGraph()
    
    #creo la lista que va a contener los grafos mergeados
    g=dict()
    
    #defino las longitudes
    L1=len(ls1)
    L2=len(ls2)
    
    #defino los instrumentos de las listas
    inst1 = [key for key in ls1.keys()]
    inst2 = [key for key in ls2.keys()]
    dif1 = list(set(inst1)-set(inst2))
    dif2 = list(set(inst2)-set(inst1))
    dif=dif1+dif2
    inter=list(set(inst1) & set(inst2))
    
    #me armo un dict con los grafos y voces que no van a aparecer
    h=dict()
    for i, inst in enumerate(dif):
        if inst in inst1:
            g= ls1['inst']
        else:
            g= ls2['inst']
            
        h.update({inst, g})

    if modelo == 'melodia': 
        instrumentos=[]
        for key1, value1 in ls1.iteritems():
            for key2, value2 in ls2.iteritems():
                #Creo los grafos que van a tener todas las canciones mergeadas pero en distintas voces
                M=nx.DiGraph()
                if key1 == key2:
                    #Uno los grafos en uno para cada voz para melodia
                    M=nx.compose(value1,value2)
                    instrumentos.append(key2)
                    g.update({key1:M})  
                            
        
    if modelo == 'ritmo': #el modelo me cambia en el tipo de grafo que me tengo que crear
        instrumentos=[]
        for key1, value1 in ls1.iteritems():
            for key2, value2 in ls2.iteritems():
                #aca es no directed
                R=nx.Graph()
                if key1 == key2:
                    #Uno los grafos en uno para cada voz para melodia
                    R=nx.compose(value1,value2)
                    instrumentos.append(key2)
                    g.update({key1:M})
    g.update(h) #le agrego los que quedaron fuera


#    if modelo == 'armonia':
#       instrumentos=[]
#        for key1, value1 in ls1.iteritems():
#            for key2, value2 in ls2.iteritems():
               #Creo los grafos que van a tener todas las canciones mergeadas pero en distintas voces
#                M=nx.DiGraph()
#                if key1 == key2:
#                    #Uno los grafos en uno para cada voz para melodia
#                    M=nx.compose(value1,value2)
#                    instrumentos.append(key2)
#                    g.update({key1:M})  
        
    return(g)

In [ ]:
#Toma dos grafos, y conecta sus notas mediante un enlace bidirigido si ambas notas fueron tocadas en simultáneo. 
#Devuelve los mismos grafos, pero conectados. Y etiquetados al instrumento que pertenecen.

def f_conect(path): #hay que pasarle una direccion para un archivo y arma un grafo de ese xml con todas las voces

    #Creo los grafos que van a tener todas las voces de un artista y todos sus temas
    M=nx.DiGraph()
    R=nx.Graph()
    #H1=nx.DiGraph()
    #H2=nx.DiGraph()

    song = msc.converter.parse(path)
    L=len(song.parts) #recorro todas las voces
    for i in range(L):
        #Uno los grafos en uno para cada voz para melodia
        m=f_xml2graph(path, nombre_parte=i, modelo='melodia');
        M=nx.compose(M,m)
        #Uno los grafos en uno para cada voz para ritmo
        r=f_xml2graph(path, nombre_parte=i, modelo='ritmo');
        R=nx.compose(R,r)
        #Para la armonia mediante f_xml2graph_armonia (cancion, index)
        #h1=f_xml2graph_armonia(paths[j], i)
        #H1=nx.compose(H1,h1)
        #Para la armonia mediante f_armon (cancion, indexes)
        #h1=f_xml2graph_armonia(paths[j], i)
        #H1=nx.compose(H1,h1)
        
    return(M,R)